# Executar um script de treinamento com SDK do Python

In [ ]:
pip show azure-ai-ml

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
pip show mlflow

In [ ]:
import pandas as pd 
print("Reading data...")
df = pd.read_csv('./Projeto_Gelato_sorvetes/dados/sorvetes_magico.csv')
df.columns = df.columns.str.strip() 
df['Data'] = pd.to_datetime(df['Data'])
df.head()

In [ ]:
print('Splitting data..')
X = df[['Temperatura']]
y = df[['Vendas']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30,random_state=0)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

In [ ]:
import mlflow
experiment_name = 'regressao_vendas_sorvete'
mlflow.set_experiment(experiment_name)

In [ ]:
mlflow.sklearn.autolog(disable=True)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

with mlflow.start_run():
    mlflow.sklearn.autolog() 

    model = LinearRegression().fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_metric("mse",mse)
    mlflow.log_metric("r2",r2)
    
    plt.figure(figsize=(6, 4))
    plt.scatter(X_test, y_test, color='blue', label='Vendas reais')
    plt.plot(X_test, y_pred, color='red', linewidth=2, label='Previsão')
    plt.xlabel('Temperatura')
    plt.ylabel('Vendas')
    plt.title('Regressão Linear')
    plt.legend()
    plt.tight_layout()
    plt.savefig("regressao.png")

    # 8. Logar imagem no MLflow
    mlflow.log_artifact("regressao.png")

    print(f"MSE: {mse:.2f}")
    print(f"R²: {r2:.2f}")